<h1> Assignment of week 3 </h1>

Explore and cluster the neighborhoods in Toronto.

Created by Tien Hoang

In [55]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
data = BeautifulSoup(source,'lxml')

In [35]:
# define the dataframe columns
column_names = ['Postalcode','Borough', 'Neighborhood'] 

# instantiate the dataframe
toronto = pd.DataFrame(columns=column_names)
toronto

,Postalcode,Borough,Neighborhood


In [36]:
#find posta code, borough, neighborhood
content = data.find('div',class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i=0
    for td in tr.find_all('td'):
        if i == 0:
            postcode=td.text
            i +=1
        elif i == 1:
            borough = td.text
            i +=1
        elif i == 2:
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode':postcode,'Borough':borough,'Neighborhood': neighborhood}, ignore_index=True)
toronto[0:5]

,Postalcode,Borough,Neighborhood
0,0,0,0
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [47]:
#clean the dataframe
toronto = toronto[toronto.Borough != 'Not assigned']
toronto = toronto[toronto.Borough != 0]
toronto.reset_index(drop=True, inplace = True)

for i in range (0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

#cleaning na, not assigned

df = df.dropna()
na = 'Not assigned'
df = df[(df.Postalcode != na ) & (df.Borough != na) & (df.Neighborhood != na)]
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [45]:
print(df.shape)

(103, 3)


In [56]:
API=''

In [57]:
latitude=[] #List to collect the latitudes
longitude=[] #List to collect the longitudes

for i in df['Postalcode']: #Iterating through Postalcodes to collect the locations data
    try:
        j='toronto,'+i
        url ="https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}".format(API,j)
        response = requests.get(url).json() # get response
        geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
        latitude.append(geographical_data['lat'])
        longitude.append(geographical_data['lng'])
    except:
        pass
df['Latitude']=latitude #Adding a column in the main dataframe for Latitude  

df['Longitude']=longitude #Adding a column in the main dataframe for Longitude 
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
